In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv3D, Flatten,MaxPooling3D,AveragePooling3D, concatenate,Input ,SpatialDropout3D,Dropout
import keras
from math import e
from sklearn.model_selection import train_test_split
from keras.models import Model
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from sklearn.model_selection import KFold
import statistics as st
from keras.utils.vis_utils import plot_model
#import visualkeras
from joblib import dump, load

In [2]:
from sklearn.metrics import confusion_matrix

In [3]:
fogData = xr.open_dataset('../../../Data/FogData/CombinedFogData_25Stations.nc')
#plt.figure(figsize=[16,8])
fogData
StackFog=fogData.fogdata.stack(a=('years','months','days'))
StackFog
dd =[];
for i in range(StackFog.years.values.shape[0]):
    dd=dd+[str(StackFog.years[i].values)+'-'+str(StackFog.months[i].values)+"-"+str(StackFog.days[i].values)]
fg = xr.Dataset({'fogdata': (('time','stations'), StackFog.values.T)}, coords={'time': pd.to_datetime(dd),'stations': fogData.stations})
yAll=fg.fogdata.sel(stations=42182).sel(time=slice('1980-12-1','2018-1-31'))
#DFog.time
#print(yAll.sel(time=slice('2018-12-1','2019-1-31')).values)
yAll


<xarray.DataArray 'fogdata' (time: 2356)>
array([0., 0., 0., ..., 1., 0., 0.])
Coordinates:
  * time      (time) datetime64[ns] 1980-12-01 1980-12-02 ... 2018-01-31
    stations  int32 42182

In [4]:
#Surface data
t2mData = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_t2m_daily_NovDecJan.nc')
d2mData = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_d2m_daily_NovDecJan.nc')
u10Data = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_u10_daily_NovDecJan.nc')
v10Data = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_v10_daily_NovDecJan.nc')
# 850 Data
tuv850Data = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_850_uvt_daily_NovDecJan.nc')
z850Data = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_850_z_daily_NovDecJan.nc')
#700 Data
w700Data = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_700_w_daily_NovDecJan.nc')
# Read Orography
OroData = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5IGP_Orography.nc')
z500Data = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_500_z_daily_NovDecJan.nc')

In [5]:
rh = (e**((17.625*(d2mData.d2m.values-273.15))/(243.04+(d2mData.d2m.values-273.15)))/e**((17.625*(t2mData.t2m.values-273.15))/(243.04+(t2mData.t2m.values-273.15))))

rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rh)}, coords={'time': t2mData.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude})


In [6]:
# Calculate wind speed 
ws = ((v10Data.v10.values**2)+(u10Data.u10.values**2))**0.5
ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)}, coords={'time': t2mData.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude})


In [7]:
#Calculate inv
inv=t2mData.t2m.values-tuv850Data.t.values
inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, coords={'time': t2mData.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude})
inv_ds.attrs
inv_ds.attrs['units']='K'
inv_ds.attrs['long_name']='t2m - t850'

In [8]:
#u shear calculation
ushear=(tuv850Data.u.values-u10Data.u10.values)/(z850Data.z.values/9.81) 
ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, coords={'time': t2mData.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude})
ushear_ds.attrs['units']='s-1'
ushear_ds.attrs['long_name']='(u10 - u850)/z850'

In [9]:
# AO data
AOData = xr.open_dataset('../../../Data/AO_EU/AO_Calculated.nc')
aoTS=AOData.AO
Darray=np.zeros((t2mData.time.shape[0],t2mData.latitude.shape[0], t2mData.longitude.shape[0]))
for t in range(aoTS.time.shape[0]) :
    Darray[t,:,:]=np.full((t2mData.latitude.shape[0], t2mData.longitude.shape[0]), aoTS[t].values)
AOData=xr.Dataset({'AO': (('time','latitude','longitude'), Darray)},
                  coords={'time': t2mData.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude}) 
# EU data
EUData = xr.open_dataset('../../../Data/AO_EU/EU.nc')
EUData.EUVal
euTS=EUData.EUVal
Darray=np.zeros((t2mData.time.shape[0],t2mData.latitude.shape[0], t2mData.longitude.shape[0]))
for t in range(euTS.time.shape[0]) :
    Darray[t,:,:]=np.full((t2mData.latitude.shape[0], t2mData.longitude.shape[0]), euTS[t].values)
EUData=xr.Dataset({'EU': (('time','latitude','longitude'), Darray)},
                  coords={'time': t2mData.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude})


In [10]:
# create mask
oro = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100))
oro.values = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100)).values/9.81
oro.attrs
oro.attrs['units']='meter'
oro.attrs['long_name']='Orography'
oro.values[oro.values>500.1]=np.NaN
mask=oro.values/oro.values



In [11]:
# 5Day and 30Day Mean
#AO
AO5D=AOData.AO.rolling(time=5).mean()

AO5DAll=AO5D[((AO5D.time.dt.month>11) | (AO5D.time.dt.month<2)) & 
             (AO5D.time.dt.year<2020)].sel(time=slice('1980-12-1','2018-1-31'),latitude=slice(35,0),
                                           longitude=slice(50,100))

#EU
EU5D=EUData.EU.rolling(time=5).mean()

EU5DAll=EU5D[((EU5D.time.dt.month>11) | (EU5D.time.dt.month<2)) & 
             (EU5D.time.dt.year<2020)].sel(time=slice('1980-12-1','2018-1-31'),latitude=slice(35,0),
                                           longitude=slice(50,100))

t1=AO5DAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
norm = StandardScaler().fit(t1)
# transform training data
t1.values = norm.transform(t1)
AO5DAll.values=t1.unstack()

t1=EU5DAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
norm = StandardScaler().fit(t1)
# transform training data
t1.values = norm.transform(t1)
EU5DAll.values=t1.unstack()

AO5DAll.values=AO5DAll.values*mask
AO5DAll.values=xr.where(np.isnan(AO5DAll.values),  0.000000000001,AO5DAll.values)

EU5DAll.values=EU5DAll.values*mask
EU5DAll.values=xr.where(np.isnan(EU5DAll.values),  0.000000000001,EU5DAll.values)

In [12]:
t2m=t2mData.t2m.shift(time=1)
ws=ws_ds.ws.shift(time=1)
rh=rh_ds.rh.shift(time=1)
inv=inv_ds.inv.shift(time=1)
w=w700Data.w.shift(time=1)
ushear=ushear_ds.ushear.shift(time=1)


In [13]:
t2mTsAll=t2m[((t2m.time.dt.month>11) | (t2m.time.dt.month<2)) & (t2m.time.dt.year<2020)].sel(time=slice('1980-12-1','2018-1-31'),latitude=slice(35,0),longitude=slice(50,100))
wsTsAll=ws[((ws.time.dt.month>11) | (ws.time.dt.month<2)) & (ws.time.dt.year<2020)].sel(time=slice('1980-12-1','2018-1-31'),latitude=slice(35,0),longitude=slice(50,100))
rhTsAll=rh[((rh.time.dt.month>11) | (rh.time.dt.month<2)) & (rh.time.dt.year<2020)].sel(time=slice('1980-12-1','2018-1-31'),latitude=slice(35,0),longitude=slice(50,100))
invTsAll=inv[((inv.time.dt.month>11) | (inv.time.dt.month<2)) & (inv.time.dt.year<2020)].sel(time=slice('1980-12-1','2018-1-31'),latitude=slice(35,0),longitude=slice(50,100))
ushearTsAll=ushear[((ushear.time.dt.month>11) | (ushear.time.dt.month<2)) & (ushear.time.dt.year<2020)].sel(time=slice('1980-12-1','2018-1-31'),latitude=slice(35,0),longitude=slice(50,100))
wTsAll=w[((w.time.dt.month>11) | (w.time.dt.month<2)) & (w.time.dt.year<2020)].sel(time=slice('1980-12-1','2018-1-31'),latitude=slice(35,0),longitude=slice(50,100))

In [14]:
t1=t2mTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
norm = StandardScaler().fit(t1)
# transform training data
t1.values = norm.transform(t1)
t2mTsAll.values=t1.unstack()

t1=wsTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
norm = StandardScaler().fit(t1)
# transform training data
t1.values = norm.transform(t1)
wsTsAll.values=t1.unstack()

t1=rhTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
norm = StandardScaler().fit(t1)
# transform training data
t1.values = norm.transform(t1)
rhTsAll.values=t1.unstack()

t1=invTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
norm = StandardScaler().fit(t1)
# transform training data
t1.values = norm.transform(t1)
invTsAll.values=t1.unstack()

t1=ushearTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
norm = StandardScaler().fit(t1)
# transform training data
t1.values = norm.transform(t1)
ushearTsAll.values=t1.unstack()


t1=wTsAll.stack(z=("latitude", "longitude"))
# fit scaler on training data
norm = StandardScaler().fit(t1)
# transform training data
t1.values = norm.transform(t1)
wTsAll.values=t1.unstack()

In [15]:
t2mTsAll.values=t2mTsAll.values*mask
wsTsAll.values=wsTsAll.values*mask
rhTsAll.values=rhTsAll.values*mask
invTsAll.values=invTsAll.values*mask
ushearTsAll.values=ushearTsAll.values*mask
wTsAll.values=wTsAll.values*mask




In [16]:
t2mTsAll.values=xr.where(np.isnan(t2mTsAll.values),  0.000000000001,t2mTsAll.values)
wsTsAll.values=xr.where(np.isnan(wsTsAll.values),  0.000000000001,wsTsAll.values)
rhTsAll.values=xr.where(np.isnan(rhTsAll.values),  0.000000000001,rhTsAll.values)
invTsAll.values=xr.where(np.isnan(invTsAll.values),  0.000000000001,invTsAll.values)
ushearTsAll.values=xr.where(np.isnan(ushearTsAll.values),  0.000000000001,ushearTsAll.values)
wTsAll.values=xr.where(np.isnan(wTsAll.values),  0.000000000001,wTsAll.values)


In [17]:
t2m_train,t2m_test,ws_train,ws_test,rh_train,rh_test,inv_train,inv_test, w_train, w_test,ushear_train, ushear_test,ao_train,ao_test,eu_train,eu_test,y_train,y_test= train_test_split( t2mTsAll,wsTsAll,rhTsAll,invTsAll, wTsAll,ushearTsAll,AO5DAll,EU5DAll,yAll,test_size=0.20, random_state=0)

In [18]:
t2mtrain=t2m_train.values
t2mtrain=t2mtrain[:,:,:,None]
t2mtrain.shape


wstrain=ws_train.values
wstrain=wstrain[:,:,:,None]
wstrain.shape

rhtrain=rh_train.values
rhtrain=rhtrain[:,:,:,None]
rhtrain.shape


invtrain=inv_train.values
invtrain=invtrain[:,:,:,None]
invtrain.shape

wtrain=w_train.values
wtrain=wtrain[:,:,:,None]
wtrain.shape

usheartrain=ushear_train.values
usheartrain=usheartrain[:,:,:,None]
usheartrain.shape

aotrain=ao_train.values
aotrain=aotrain[:,:,:,None]
aotrain.shape

eutrain=eu_train.values
eutrain=eutrain[:,:,:,None]
eutrain.shape


(1884, 18, 26, 1)

In [19]:
t2mtest=t2m_test.values
t2mtest=t2mtest[:,:,:,None]
t2mtest.shape


wstest=ws_test.values
wstest=wstest[:,:,:,None]
wstest.shape

rhtest=rh_test.values
rhtest=rhtest[:,:,:,None]
rhtest.shape


invtest=inv_test.values
invtest=invtest[:,:,:,None]
invtest.shape

wtest=w_test.values
wtest=wtest[:,:,:,None]
wtest.shape

usheartest=ushear_test.values
usheartest=usheartest[:,:,:,None]
usheartest.shape

aotest=ao_test.values
aotest=aotest[:,:,:,None]
aotest.shape

eutest=eu_test.values
eutest=eutest[:,:,:,None]
eutest.shape


(472, 18, 26, 1)

In [20]:
ytrain=y_train.values
ytrain=ytrain[:,None]
ytrain.shape
ytest=y_test.values
ytest=ytest[:,None]
ytest.shape

(472, 1)

In [21]:
#X_test=np.array([t2mtest,rhtest,wstest,invtest,wtest,usheartest,aotest,eutest])
#X_test=np.array([t2mtest,rhtest,wstest,invtest,wtest,usheartest,aotest,eutest,ssttest])
X_test=np.array([t2mtest,rhtest,wstest,invtest,wtest,usheartest,aotest,eutest])
X_test.shape

#X_train=np.array([t2mtrain,rhtrain,wstrain,invtrain,wtrain,usheartrain,aotrain,eutrain])
#X_train=np.array([t2mtrain,rhtrain,wstrain,invtrain,wtrain,usheartrain,aotrain,eutrain,ssttrain])
X_train=np.array([t2mtrain,rhtrain,wstrain,invtrain,wtrain,usheartrain,aotrain,eutrain])
X_train.shape

(8, 1884, 18, 26, 1)

In [22]:
X_train_reshape = np.einsum('lkija->klija',X_train)
X_train_reshape.shape

X_test_reshape = np.einsum('lkija->klija',X_test)
X_test_reshape.shape

(472, 8, 18, 26, 1)

In [26]:
    from numpy.random import seed
    seed(123)
    from tensorflow import random
    random.set_seed(123)
    model = Sequential()
    model.add(Conv3D(16, kernel_size=3, activation='relu',padding='same',input_shape=(X_train_reshape.shape[1],X_train_reshape.shape[2],X_train_reshape.shape[3],1)),)
    model.add(AveragePooling3D(pool_size=2,padding='same'))
    model.add(Conv3D(32, kernel_size=3, activation='relu',padding='same'))
    model.add(AveragePooling3D(pool_size=2,padding='same'))
    model.add(Dropout(0.2))
    model.add(Conv3D(64, kernel_size=3, activation='relu',padding='same'))
    model.add(AveragePooling3D(pool_size=2,padding='same'))
    model.add(Dropout(0.2))


    model.add(Flatten())

    model.add(Dense(32, activation='relu'))
    model.add(Dense(units=1, activation = 'relu'))

    #rm = keras.optimizers.RMSprop(lr=0.01)
    model.compile(optimizer=keras.optimizers.RMSprop(lr=0.001), loss='binary_crossentropy',  metrics='accuracy')
    history = model.fit(X_train_reshape, y_train, epochs=20)
    yy_test = model.predict(X_test_reshape)
    
   # test_corr = np.corrcoef(yy_test[:,0],y_test)[0,1]
   # plt.title('corr = %f'%test_corr)

    #plt.plot(yy_test,y_test,'+')
   
    


Epoch 1/20
59/59 [==============================] - 2s 19ms/step - loss: 2.6630 - accuracy: 0.4614
Epoch 2/20
59/59 [==============================] - 1s 19ms/step - loss: 0.7788 - accuracy: 0.5479
Epoch 3/20
59/59 [==============================] - 1s 19ms/step - loss: 0.7159 - accuracy: 0.6039
Epoch 4/20
59/59 [==============================] - 1s 19ms/step - loss: 0.6891 - accuracy: 0.5884
Epoch 5/20
59/59 [==============================] - 1s 19ms/step - loss: 0.6789 - accuracy: 0.6184
Epoch 6/20
59/59 [==============================] - 1s 18ms/step - loss: 0.6730 - accuracy: 0.6254
Epoch 7/20
59/59 [==============================] - 1s 19ms/step - loss: 0.6136 - accuracy: 0.6642
Epoch 8/20
59/59 [==============================] - 1s 19ms/step - loss: 0.6553 - accuracy: 0.6819
Epoch 9/20
59/59 [==============================] - 1s 19ms/step - loss: 0.5739 - accuracy: 0.7029
Epoch 10/20
59/59 [==============================] - 1s 19ms/step - loss: 0.5521 - accuracy: 0.7250
Epoch 11/

In [27]:
print(yy_test[:,0])

yf=[]

for y in yy_test[:,0]:
    if y >= 0.5: 
        yf.append(1)
    else: 
        yf.append(0)
    
yfArray_ds=xr.Dataset({'yfArray': (('time'), yf)}, coords={'time':y_test.time.values})
    
print(yfArray_ds.yfArray)
print(y_test)


tn, fp, fn, tp = confusion_matrix(y_test.values, yfArray_ds.yfArray.values).ravel()

print(tn, fp, fn, tp)

[6.46961749e-01 4.71001655e-01 2.92034239e-01 5.70816159e-01
 4.85847294e-01 5.80180407e-01 1.05320191e+00 1.01678598e+00
 1.21158212e-01 3.56689811e-01 9.45816517e-01 7.33161449e-01
 7.71246493e-01 5.11941254e-01 5.52878320e-01 1.39148861e-01
 4.81895566e-01 4.53463435e-01 0.00000000e+00 2.23370254e-01
 1.22742727e-03 9.53620374e-01 6.41385496e-01 8.29246342e-01
 7.00808704e-01 6.50514305e-01 1.52096808e+00 9.19211745e-01
 3.18839103e-01 6.03190720e-01 3.03955674e-01 4.57723886e-01
 9.03963864e-01 5.28294504e-01 3.99865985e-01 4.52627987e-01
 5.02306998e-01 2.74357468e-01 7.60245502e-01 4.17364687e-01
 4.53212947e-01 1.05204880e+00 1.65665746e-01 6.33402288e-01
 2.11080089e-01 3.29012424e-01 3.01023722e-01 4.27949786e-01
 6.67113662e-01 7.16367722e-01 8.65519345e-01 8.16830099e-01
 0.00000000e+00 6.49479389e-01 5.09238482e-01 5.42852998e-01
 7.11614728e-01 2.88197756e-01 4.28453892e-01 2.62883484e-01
 4.71414387e-01 8.04044545e-01 6.24791443e-01 5.51315725e-01
 8.52851212e-01 8.153229

In [28]:
# save model and architecture to single file
model.save("modelDFogCNN.h5")
print("Saved model to disk")

Saved model to disk
